In [31]:
import os, re, pickle
from collections import Counter

import pandas as pd

In [32]:
path = '../../../data/MEDS/Demo Dataset/Datapoint Classifier/BS'
filename = 'Data_point_BS.csv'
output_filename = filename.split('.')[0] + '-ulmfit.csv'
df = pd.read_csv(os.path.join(path, filename))
df.head()

,CompanyId,crn,matched_crn,SectorName,IndustryName,IndustryTemplateCode,MsId
0,0C000009IN,cash and bank balaaces,cash and bank balance,communication services,telecom services,n,IFBS000300
1,0C000009IN,borrowings,borrowing,communication services,telecom services,n,IFBS000480
2,0C000009IN,deferred tax liabilities,deferred tax liability,communication services,telecom services,n,IFBS000710
3,0C000009IN,borrowings,borrowing,communication services,telecom services,n,IFBS001290
4,0C000009IN,other current liabilities,other current liability,communication services,telecom services,n,IFBS001570


In [33]:
df = df[['MsId', 'crn']].rename(columns={'crn': 'text', 'MsId': 'label'})
df = df[df.text == df.text]
df.head()

,label,text
0,IFBS000300,cash and bank balaaces
1,IFBS000480,borrowings
2,IFBS000710,deferred tax liabilities
3,IFBS001290,borrowings
4,IFBS001570,other current liabilities


In [34]:
text = ' '.join(df['text'].values)
text[:200]

'cash and bank balaaces borrowings deferred tax liabilities borrowings other current liabilities intangible assets under development capital work in progres s goodwill on consolidation other non curren'

In [35]:
# Loading english_words
with open ('english_words', 'rb') as f:
    english_words = pickle.load(f)
    
def words(sentence): 
        all_words = re.findall(r'\w+', sentence.lower())
        return [w for w in all_words if len(w)>2]

count_dictionary = Counter(words(text))

In [36]:
def correction(word):
    "Most probable spelling correction for word."
    
    def P(word, N=sum(count_dictionary.values())): 
        "Probability of `word`."
        return count_dictionary[word] / N

    def candidates(word): 
        "Generate possible spelling corrections for word."
        return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

    def known(words): 
        "The subset of `words` that appear in the dictionary of count_dictionary."
        return set(w for w in words if w in count_dictionary)

    def edits1(word):
        "All edits that are one edit away from `word`."
        letters    = 'abcdefghijklmnopqrstuvwxyz'
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
        inserts    = [L + c + R               for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(word): 
        "All edits that are two edits away from `word`."
        return (e2 for e1 in edits1(word) for e2 in edits1(e1))

    return max(candidates(word), key=P)

def correct_sentence(sentence):
    '''Returns a corrected sentence'''
#     sentence = text_corrector(sentence)
#     return ' '.join(correction(w.replace('\'s','').replace('\'','')) for w in sentence.split())
    return ' '.join(correction((w)) if w not in english_words else w for w in sentence.split() )
    
words = ['csh', 'bank', 'balaaces', 'borowings', 'defered', 'taxx', 'liabiities', 'borowings']
print(correct_sentence(' '.join(words)))

cash bank balaaces borrowings defered tax liabiities borrowings


In [30]:
from textblob import TextBlob
line = ' '.join(words)
TextBlob(line).correct()

TextBlob("cash bank balances borrowings deferred tax liabilities borrowings")